In [2]:
# Env
import os
from dotenv import load_dotenv
if not load_dotenv() or 'HOPSWORKS_API_KEY' not in os.environ:
    print("Please provide a .env file add HOPSWORKS_API_KEY to it.")
    exit(1)

# Imports
import seaborn as sns
sns.set_palette('husl')
import matplotlib.pyplot as plt
import hopsworks
import pandas as pd

In [3]:
project = hopsworks.login()
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/201877
Connected. Call `.close()` to terminate connection gracefully.


In [4]:
vessel_df = pd.read_csv("./backfill/vessel_data.csv")
vessel_df

,ship_id,ship_name,latitude,longitude,time,dim_a,dim_b,dim_c,dim_d,length,width,eta_day,eta_hour,eta_minute,eta_month,destination,ship_type,ship_heading,ship_speed
0,304997000,WILSON DIRDAL,60.125667,19.102500,2024-01-04 09:14:59.591590409 +0000 UTC,75.0,13.0,5.0,8.0,88.0,13.0,4.0,11.0,0.0,1.0,FOR ORDER,79.0,138.0,7.2
1,236111925,SOUND SOLUTION,59.354483,17.964942,2024-01-04 09:15:21.493471037 +0000 UTC,12.0,13.0,4.0,6.0,25.0,10.0,28.0,9.0,0.0,12.0,STOCKHOLM,52.0,254.0,0.0
2,265518880,RESCUE BURRE,57.418768,16.677853,2024-01-04 09:15:13.416253033 +0000 UTC,6.0,6.0,2.0,2.0,12.0,4.0,31.0,23.0,59.0,12.0,SWEDISH SAR VESSEL,51.0,87.0,0.0
3,265585310,JUPITER,59.388627,18.327317,2024-01-04 09:15:12.400625784 +0000 UTC,43.0,43.0,9.0,9.0,86.0,18.0,0.0,24.0,60.0,0.0,LJUSTERO<>OSTANA :-),65.0,147.0,0.0
4,265647200,ASKHOLMEN,59.538273,18.750687,2024-01-04 09:15:29.444613983 +0000 UTC,15.0,6.0,3.0,3.0,21.0,6.0,10.0,12.0,0.0,8.0,KLINTSUNDET,99.0,511.0,0.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104450,265532910,NaN,59.328862,18.088587,2024-01-08 18:14:47.305849861 +0000 UTC,20.0,10.0,2.0,4.0,30.0,6.0,NaN,NaN,NaN,NaN,NaN,36.0,511.0,0.0
104451,265650810,NaN,59.354137,18.050013,2024-01-08 18:16:40.183182999 +0000 UTC,4.0,8.0,2.0,2.0,12.0,4.0,NaN,NaN,NaN,NaN,NaN,37.0,511.0,0.0
104452,265702750,ESTELLE,58.488998,16.942075,2024-01-08 18:18:57.083635955 +0000 UTC,4.0,9.0,2.0,2.0,13.0,4.0,NaN,NaN,NaN,NaN,NaN,60.0,240.0,0.0
104453,230985650,OSMERUS,60.520253,21.659818,2024-01-08 18:18:13.852976473 +0000 UTC,5.0,10.0,4.0,1.0,15.0,5.0,7.0,20.0,30.0,12.0,TUOMARAINEN,30.0,94.0,0.1


In [5]:
vessel_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104455 entries, 0 to 104454
Data columns (total 19 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   ship_id       104455 non-null  int64  
 1   ship_name     102152 non-null  object 
 2   latitude      104455 non-null  float64
 3   longitude     104455 non-null  float64
 4   time          104455 non-null  object 
 5   dim_a         104455 non-null  float64
 6   dim_b         104455 non-null  float64
 7   dim_c         104455 non-null  float64
 8   dim_d         104455 non-null  float64
 9   length        104455 non-null  float64
 10  width         104455 non-null  float64
 11  eta_day       98391 non-null   float64
 12  eta_hour      98391 non-null   float64
 13  eta_minute    98391 non-null   float64
 14  eta_month     98391 non-null   float64
 15  destination   82371 non-null   object 
 16  ship_type     104455 non-null  float64
 17  ship_heading  104455 non-null  float64
 18  ship

In [7]:
vessel_df.isnull().sum()

ship_id             0
ship_name        2303
latitude            0
longitude           0
time                0
dim_a               0
dim_b               0
dim_c               0
dim_d               0
length              0
width               0
eta_day          6064
eta_hour         6064
eta_minute       6064
eta_month        6064
destination     22084
ship_type           0
ship_heading        0
ship_speed          0
dtype: int64

In [14]:
# Drop columns with null/nan
filtered_vessel_df = vessel_df.drop(columns=['ship_name', 'eta_day', 'eta_hour', 'eta_minute', 'eta_month', 'destination'])
filtered_vessel_df

,ship_id,latitude,longitude,time,dim_a,dim_b,dim_c,dim_d,length,width,ship_type,ship_heading,ship_speed
0,304997000,60.125667,19.102500,2024-01-04 09:14:59.591590409 +0000 UTC,75.0,13.0,5.0,8.0,88.0,13.0,79.0,138.0,7.2
1,236111925,59.354483,17.964942,2024-01-04 09:15:21.493471037 +0000 UTC,12.0,13.0,4.0,6.0,25.0,10.0,52.0,254.0,0.0
2,265518880,57.418768,16.677853,2024-01-04 09:15:13.416253033 +0000 UTC,6.0,6.0,2.0,2.0,12.0,4.0,51.0,87.0,0.0
3,265585310,59.388627,18.327317,2024-01-04 09:15:12.400625784 +0000 UTC,43.0,43.0,9.0,9.0,86.0,18.0,65.0,147.0,0.0
4,265647200,59.538273,18.750687,2024-01-04 09:15:29.444613983 +0000 UTC,15.0,6.0,3.0,3.0,21.0,6.0,99.0,511.0,0.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
104450,265532910,59.328862,18.088587,2024-01-08 18:14:47.305849861 +0000 UTC,20.0,10.0,2.0,4.0,30.0,6.0,36.0,511.0,0.0
104451,265650810,59.354137,18.050013,2024-01-08 18:16:40.183182999 +0000 UTC,4.0,8.0,2.0,2.0,12.0,4.0,37.0,511.0,0.0
104452,265702750,58.488998,16.942075,2024-01-08 18:18:57.083635955 +0000 UTC,4.0,9.0,2.0,2.0,13.0,4.0,60.0,240.0,0.0
104453,230985650,60.520253,21.659818,2024-01-08 18:18:13.852976473 +0000 UTC,5.0,10.0,4.0,1.0,15.0,5.0,30.0,94.0,0.1


In [18]:
filtered_vessel_df.describe(percentiles=[0.25, 0.50, 0.90]).T

,count,mean,std,min,25%,50%,90%,max
ship_id,104455.0,2.628631e+08,4.232936e+07,2.091510e+08,2.558059e+08,2.655305e+08,2.658178e+08,6.360221e+08
latitude,104455.0,5.937026e+01,7.546946e-01,5.711989e+01,5.905233e+01,5.933103e+01,6.053499e+01,6.134453e+01
longitude,104455.0,1.823924e+01,1.206846e+00,1.505748e+01,1.763130e+01,1.809918e+01,1.925817e+01,2.176120e+01
dim_a,104455.0,3.155984e+01,3.963790e+01,1.000000e+00,8.000000e+00,1.200000e+01,9.400000e+01,5.060000e+02
dim_b,104455.0,2.432117e+01,3.514815e+01,0.000000e+00,8.000000e+00,1.500000e+01,4.100000e+01,4.260000e+02
dim_c,104455.0,5.375367e+00,4.719525e+00,0.000000e+00,2.000000e+00,4.000000e+00,1.100000e+01,6.300000e+01
dim_d,104455.0,5.219827e+00,4.158590e+00,0.000000e+00,2.000000e+00,4.000000e+00,1.100000e+01,6.100000e+01
length,104455.0,5.588066e+01,5.476156e+01,3.000000e+00,1.800000e+01,3.100000e+01,1.400000e+02,8.870000e+02
width,104455.0,1.059479e+01,7.474291e+00,1.000000e+00,5.000000e+00,8.000000e+00,2.200000e+01,9.200000e+01
ship_type,104455.0,6.029368e+01,1.521779e+01,6.000000e+00,5.100000e+01,6.000000e+01,7.900000e+01,2.520000e+02


### Insert our Vessel DataFrame into a FeatureGroup

In [20]:
vessel_fg = fs.get_or_create_feature_group(
    name="vessel",
    version=1,
    primary_key=["ship_id", "time"], 
    description="Vesel dataset")
vessel_fg.insert(filtered_vessel_df)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/201877/fs/201796/fg/398682


Uploading Dataframe: 0.00% |          | Rows 0/104455 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: vessel_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/201877/jobs/named/vessel_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7feb714eb970>, None)